In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding, Dense, Input, Flatten, concatenate,LSTM,Dropout
from tensorflow.keras.models import Model
import pandas as pd

import numpy as np
from sklearn.preprocessing import OrdinalEncoder
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/trip_advisor_data.csv')
data.head()

,overall,cleanliness,location,rooms,service,value,title,text,username,num_cities,num_helpful_votes,num_reviews,num_type_reviews,id,date_stayed,offering_id,date,via_mobile,user_id
0,4.0,5.0,5.0,5.0,4.0,3.0,“Its the best of the Andaz Brand in the US....”,I have stayed at each of the US Andaz properti...,JamesE339,34.0,0,104.0,49.0,147612823,December 2012,1762573,"December 17, 2012",False,BA524A238B1171206691A6CC3F28F266
1,4.0,5.0,5.0,5.0,4.0,5.0,“Stunningly Wonderful!”,Other hotels in NYC that are space challenged ...,romadaro,70.0,5,103.0,85.0,147531970,December 2012,1762573,"December 16, 2012",False,C81AB7D49D98FA410EA191E15F427BEC
2,5.0,5.0,4.0,5.0,5.0,3.0,“Unexpectedly good”,"Campus hotels are a little unpredictable, but ...",Grotshops,35.0,0,96.0,51.0,140624089,September 2012,108038,"September 18, 2012",False,4C8EFD12CA2F03E9D85709814BB16501
3,4.0,4.0,3.0,4.0,4.0,4.0,"“Pleasant enough, but a bit overrated”",For our most recent ever-so-brief New York vis...,pillowsofwanderlust,20.0,2,57.0,49.0,145940048,November 2012,1776857,"November 22, 2012",False,3F88F8D4F220EFE6418ED0B22AAF03E6
4,4.0,4.0,4.0,5.0,4.0,4.0,“nice suprise”,Booked this hotel thinking it was another Hilt...,beatricepugzilla,28.0,2,41.0,36.0,127060765,March 2012,108038,"April 1, 2012",False,A90E45A9CAD3E818327172C16C27BFB1


In [ ]:
data = data[data.location.isin(['0.0','1.0','2.0','3.0','4.0','5.0'])]

In [ ]:
cls = ['cleanliness','location','rooms','service','value']
for cl in cls:
  data[cl] = np.asarray(data[cl]).astype(np.float64)

for cl in cls:
  print(cls,data[data[cl]==1.0].shape,data[data[cl]==2.0].shape,data[data[cl]==3.0].shape,data[data[cl]==4.0].shape,data[data[cl]==5.0].shape)

['cleanliness', 'location', 'rooms', 'service', 'value'] (307, 19) (349, 19) (991, 19) (2497, 19) (5524, 19)
['cleanliness', 'location', 'rooms', 'service', 'value'] (137, 19) (251, 19) (934, 19) (2360, 19) (5986, 19)
['cleanliness', 'location', 'rooms', 'service', 'value'] (413, 19) (579, 19) (1556, 19) (2978, 19) (4142, 19)
['cleanliness', 'location', 'rooms', 'service', 'value'] (486, 19) (446, 19) (1160, 19) (2341, 19) (5235, 19)
['cleanliness', 'location', 'rooms', 'service', 'value'] (522, 19) (610, 19) (1586, 19) (3009, 19) (3941, 19)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
ord_enc = OrdinalEncoder()
data['offering_id'] =ord_enc.fit_transform(data['offering_id'].values.reshape(-1, 1)).reshape(1,-1)[0]
data['user_id'] =ord_enc.fit_transform(data['user_id'].values.reshape(-1, 1)).reshape(1,-1)[0]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
import re
def remove_punc(line):
  line = re.sub('[0-9@]', '',line)
  line = re.sub('[#[]', '',line)
  line = re.sub(']', '',line)
  line = re.sub('[\'":,.?!&)(]', '',line)
  line = line.rstrip().lstrip()
  return line

data['text'] = data['text'].apply(lambda x : remove_punc(x))
data.text.values

array(['I have stayed at each of the US Andaz properties and this one is the best of them all MUCH better than the West Hollywood property where Brand standards are slowly fading This is the shining star here in the States\nMy room suite on the th floor was fantastic as usual Perfectly maintained very comfortable great bathrooms best in NYC The bathroom is huge with large shower room and multiple shower heads Double lavatories flank one wall Huge\nTHe bed is always perfect Televisions in both rooms were perfect for my limited time spend in the suite Nice desk for work area and in room dining \nEvery room has a fridge with complimentary soft drinks snacks water sparkling water milk and juices Nice touch Suite has full size refrigerator with icemaker and also features a microwave sink and dishwasher drawerall nicely concealed behind beautiful cabinetry Really great for a longer stay\nI do have  complaints First the elevators The waits were sometimes ridiculous  elevators serve the main t

In [ ]:

import nltk
from nltk.corpus import stopwords
stop_w = stopwords.words('english')

data['text'] = data['text'].apply(lambda x : ' '.join([w.lower() for w in x.split(' ') if w.lower() not in stop_w ]))
data['text'] = data['text'].apply(lambda x : ' '.join([t for t in x.split(' ') if len(t)>0]))
    
data.text.values

array(['stayed us andaz properties one best much better west hollywood property brand standards slowly fading shining star states\nmy room suite th floor fantastic usual perfectly maintained comfortable great bathrooms best nyc bathroom huge large shower room multiple shower heads double lavatories flank one wall huge\nthe bed always perfect televisions rooms perfect limited time spend suite nice desk work area room dining \nevery room fridge complimentary soft drinks snacks water sparkling water milk juices nice touch suite full size refrigerator icemaker also features microwave sink dishwasher drawerall nicely concealed behind beautiful cabinetry really great longer stay\ni complaints first elevators waits sometimes ridiculous elevators serve main tower rather limited number rooms newer hotel service elevator continually surprised waiting elevator many stops made housekeeping enter carts room service carts trays etc could hyatt skip service elevator new property really stinks timesit

In [ ]:
from tensorflow.keras.regularizers import L1L2
from tensorflow.keras.layers import Dense, LSTM, Embedding,Bidirectional
import tensorflow
from tensorflow.compat.v1.keras.layers import CuDNNLSTM
def mod():
  sent_inp = Input(shape=(150, 768))
  X = Bidirectional(CuDNNLSTM(512, return_sequences=True))(sent_inp)
  
  #X = Bidirectional(CuDNNLSTM(100,return_sequences=True))(X)
  #X = Dropout(0.4)(X)
  X = Bidirectional(CuDNNLSTM(256,return_sequences=True))(X)
  X = Dropout(0.3)(X)
  X = Bidirectional(CuDNNLSTM(128))(X)
  X = Dropout(0.2)(X)
  X = Dense(32,activation='relu')(X) 
  out = Dense(5,activation='sigmoid')(X)

 
  


  
  
  
  
  model = Model(inputs=sent_inp, 
                  outputs=out)
  
  return model


In [ ]:
model_sent = mod()

In [ ]:
from tensorflow.keras.models import load_model
model_sent = load_model('/content/drive/MyDrive/model_multi_sentimentv2.h5')

In [ ]:
!pip install --upgrade transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 29.0 MB/s 
     |████████████████████████████████| 6.6 MB 53.7 MB/s 
     |████████████████████████████████| 596 kB 59.6 MB/s 
     |████████████████████████████████| 101 kB 14.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from transformers import BertTokenizer, TFBertModel,pipeline
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertModel.from_pretrained("bert-base-uncased")


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/511M [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
texts = data.text.values
texts

array(['stayed us andaz properties one best much better west hollywood property brand standards slowly fading shining star states\nmy room suite th floor fantastic usual perfectly maintained comfortable great bathrooms best nyc bathroom huge large shower room multiple shower heads double lavatories flank one wall huge\nthe bed always perfect televisions rooms perfect limited time spend suite nice desk work area room dining \nevery room fridge complimentary soft drinks snacks water sparkling water milk juices nice touch suite full size refrigerator icemaker also features microwave sink dishwasher drawerall nicely concealed behind beautiful cabinetry really great longer stay\ni complaints first elevators waits sometimes ridiculous elevators serve main tower rather limited number rooms newer hotel service elevator continually surprised waiting elevator many stops made housekeeping enter carts room service carts trays etc could hyatt skip service elevator new property really stinks timesit

In [ ]:
texts = data.text.values
i=0
pred = []
for text in texts :
  i+=1
  if i%1000==0:
    print(len(texts)-i)
  encoded_input = tokenizer([text], return_tensors='tf',max_length = 150,  # maximum length of a sentence  (TODO Figure the longest passage length)
      truncation = 'longest_first',padding='max_length'
    )
    
  output = model(encoded_input)
  prd = model_sent.predict(output['last_hidden_state'])[0]
  
  ls = []
  for j in range(5):
    ls.append(prd[j])
  pred.append(ls.copy())
  

8668
7668
6668
5668
4668
3668
2668
1668
668


In [ ]:
pred

[[0.9989712, 0.99548644, 0.9973354, 0.9978877, 0.99654937],
 [0.9996897, 0.997335, 0.9990748, 0.99916685, 0.9987747],
 [0.9969001, 0.98979986, 0.99046516, 0.99454427, 0.9875053],
 [0.99936575, 0.9952212, 0.99807584, 0.99840873, 0.9976725],
 [0.99610317, 0.9877458, 0.9883279, 0.99307317, 0.9850794],
 [0.99961036, 0.9969414, 0.998818, 0.998955, 0.99850106],
 [0.99835455, 0.99298805, 0.9946865, 0.9971253, 0.9937516],
 [0.9994868, 0.99637645, 0.99846506, 0.9986755, 0.99805945],
 [0.996161, 0.9869227, 0.9887765, 0.9925218, 0.98662543],
 [0.99977285, 0.9979085, 0.99929523, 0.9993974, 0.99911827],
 [0.99708444, 0.98943734, 0.9903008, 0.9940246, 0.98853314],
 [0.9992415, 0.99537224, 0.99780065, 0.9981238, 0.9972102],
 [0.17780656, 0.49925914, 0.0334201, 0.029822255, 0.03141656],
 [0.9994166, 0.9962035, 0.99831104, 0.9985262, 0.99783677],
 [0.9425899, 0.887938, 0.82479125, 0.8926945, 0.82630175],
 [0.9981279, 0.9929039, 0.99443495, 0.99667287, 0.99289083],
 [0.99968266, 0.99729025, 0.99907887, 

In [ ]:
pred = np.array(pred)
data['cleanliness_sent'] = [1 if prd >= 0.5 else 0 for prd in pred[:,0]]
data['location_sent'] = [1 if prd >= 0.5 else 0 for prd in pred[:,1]]
data['rooms_sent'] = [1 if prd >= 0.5 else 0 for prd in pred[:,2]]
data['service_sent'] = [1 if prd >= 0.5 else 0 for prd in pred[:,3]]
data['value_sent'] = [1 if prd >= 0.5 else 0 for prd in pred[:,4]]

In [ ]:
data.to_csv('trip_data_multi.csv',index=False)

In [ ]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(data,test_size=0.2,random_state=123)
train_data.to_csv('data_multi_sent.csv',index=False)

In [ ]:
data

,overall,cleanliness,location,rooms,service,value,title,text,username,num_cities,...,date_stayed,offering_id,date,via_mobile,user_id,cleanliness_sent,location_sent,rooms_sent,service_sent,value_sent
0,4.0,5.0,5.0,5.0,4.0,3.0,“Its the best of the Andaz Brand in the US....”,stayed us andaz properties one best much bette...,JamesE339,34.0,...,December 2012,1927.0,"December 17, 2012",False,626.0,1,1,1,1,1
1,4.0,5.0,5.0,5.0,4.0,5.0,“Stunningly Wonderful!”,hotels nyc space challenged would well take no...,romadaro,70.0,...,December 2012,1927.0,"December 16, 2012",False,677.0,1,1,1,1,1
2,5.0,5.0,4.0,5.0,5.0,3.0,“Unexpectedly good”,campus hotels little unpredictable one star ro...,Grotshops,35.0,...,September 2012,849.0,"September 18, 2012",False,271.0,1,1,1,1,1
3,4.0,4.0,3.0,4.0,4.0,4.0,"“Pleasant enough, but a bit overrated”",recent ever-so-brief new york visit days night...,pillowsofwanderlust,20.0,...,November 2012,1929.0,"November 22, 2012",False,224.0,1,1,1,1,1
4,4.0,4.0,4.0,5.0,4.0,4.0,“nice suprise”,booked hotel thinking another hilton property ...,beatricepugzilla,28.0,...,March 2012,849.0,"April 1, 2012",False,583.0,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10045,5.0,5.0,5.0,4.0,5.0,3.0,"“Great Location, would stay again”",nice hotel decor might everyones taste quite d...,CityGirl69,6.0,...,February 2009,353.0,"February 27, 2009",False,824.0,1,1,1,1,1
10046,4.0,5.0,5.0,5.0,4.0,4.0,“A nice hotel in a nice location - with staff ...,initial confusion part sent us palomar arlingt...,managementtrainer1,8.0,...,October 2008,353.0,"November 12, 2008",False,376.0,1,1,1,1,1
10047,5.0,4.0,3.0,5.0,4.0,4.0,“Hotel Palomar gets everything right”,nicely designed hotel quiet clean decent sized...,Mattonholiday,6.0,...,March 2008,353.0,"March 31, 2008",False,64.0,1,1,1,1,1
10048,3.0,4.0,5.0,4.0,4.0,4.0,“Friendly staff-noisy doors everywhere”,pros\n-friendly staff particularly doormen val...,larklaw,43.0,...,November 2007,353.0,"November 27, 2007",False,470.0,1,1,1,1,1


In [ ]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5,random_state=24,shuffle=True)

In [ ]:
data.head()

,overall,cleanliness,location,rooms,service,value,title,text,username,num_cities,...,date_stayed,offering_id,date,via_mobile,user_id,cleanliness_sent,location_sent,rooms_sent,service_sent,value_sent
0,4.0,5.0,5.0,5.0,4.0,3.0,“Its the best of the Andaz Brand in the US....”,stayed us andaz properties one best much bette...,JamesE339,34.0,...,December 2012,1927.0,"December 17, 2012",False,626.0,1,1,1,1,1
1,4.0,5.0,5.0,5.0,4.0,5.0,“Stunningly Wonderful!”,hotels nyc space challenged would well take no...,romadaro,70.0,...,December 2012,1927.0,"December 16, 2012",False,677.0,1,1,1,1,1
2,5.0,5.0,4.0,5.0,5.0,3.0,“Unexpectedly good”,campus hotels little unpredictable one star ro...,Grotshops,35.0,...,September 2012,849.0,"September 18, 2012",False,271.0,1,1,1,1,1
3,4.0,4.0,3.0,4.0,4.0,4.0,"“Pleasant enough, but a bit overrated”",recent ever-so-brief new york visit days night...,pillowsofwanderlust,20.0,...,November 2012,1929.0,"November 22, 2012",False,224.0,1,1,1,1,1
4,4.0,4.0,4.0,5.0,4.0,4.0,“nice suprise”,booked hotel thinking another hilton property ...,beatricepugzilla,28.0,...,March 2012,849.0,"April 1, 2012",False,583.0,1,1,1,1,1


In [ ]:
f=0
for train_index, test_index in kf.split(data):
  print('fold number : ',f)
  f+=1
  train_data , test_data = data.iloc[train_index,:] , data.iloc[test_index,:]
  train_data.to_csv('/content/drive/MyDrive/Trip_multi_fold_with_sentiment_fl'+str(f)+'_train.csv',index=False)
  test_data.to_csv('/content/drive/MyDrive/Trip_multi_fold_with_sentiment_fl'+str(f) +'_test.csv',index=False)

fold number :  0
fold number :  1
fold number :  2
fold number :  3
fold number :  4


In [ ]:
train_data

,overall,cleanliness,location,rooms,service,value,title,text,username,num_cities,...,date_stayed,offering_id,date,via_mobile,user_id,cleanliness_sent,location_sent,rooms_sent,service_sent,value_sent
1,4.0,5.0,5.0,5.0,4.0,5.0,“Stunningly Wonderful!”,hotels nyc space challenged would well take no...,romadaro,70.0,...,December 2012,1927.0,"December 16, 2012",False,677.0,1,1,1,1,1
2,5.0,5.0,4.0,5.0,5.0,3.0,“Unexpectedly good”,campus hotels little unpredictable one star ro...,Grotshops,35.0,...,September 2012,849.0,"September 18, 2012",False,271.0,1,1,1,1,1
3,4.0,4.0,3.0,4.0,4.0,4.0,"“Pleasant enough, but a bit overrated”",recent ever-so-brief new york visit days night...,pillowsofwanderlust,20.0,...,November 2012,1929.0,"November 22, 2012",False,224.0,1,1,1,1,1
6,4.0,4.0,4.0,3.0,5.0,4.0,“Service was outstanding...dont let decor get ...,like everyone posted physical structure dated ...,bobolito,28.0,...,October 2012,682.0,"October 25, 2012",False,818.0,1,1,1,1,1
8,5.0,5.0,5.0,5.0,5.0,5.0,“Outstanding Service and facility! If you want...,outstanding great staff service facility conve...,Tmayfield1964,NaN,...,September 2012,682.0,"September 13, 2012",False,412.0,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10044,3.0,4.0,4.0,4.0,2.0,5.0,“Great Location. Terrible Service”,friends booked hotel weekend evening night bir...,reeseoreo,NaN,...,March 2009,353.0,"March 9, 2009",False,173.0,1,1,1,1,1
10045,5.0,5.0,5.0,4.0,5.0,3.0,"“Great Location, would stay again”",nice hotel decor might everyones taste quite d...,CityGirl69,6.0,...,February 2009,353.0,"February 27, 2009",False,824.0,1,1,1,1,1
10047,5.0,4.0,3.0,5.0,4.0,4.0,“Hotel Palomar gets everything right”,nicely designed hotel quiet clean decent sized...,Mattonholiday,6.0,...,March 2008,353.0,"March 31, 2008",False,64.0,1,1,1,1,1
10048,3.0,4.0,5.0,4.0,4.0,4.0,“Friendly staff-noisy doors everywhere”,pros\n-friendly staff particularly doormen val...,larklaw,43.0,...,November 2007,353.0,"November 27, 2007",False,470.0,1,1,1,1,1


In [ ]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=5)

f=0
for train_index, test_index in skf.split(data,data.user_id):
  print('fold number : ',f)
  f+=1
  train_data , test_data = data.iloc[train_index,:] , data.iloc[test_index,:]
  train_data.to_csv('/content/drive/MyDrive/Trip_multi_fold_with_sentiment_fl_strtified'+str(f)+'_train.csv',index=False)
  test_data.to_csv('/content/drive/MyDrive/Trip_multi_fold_with_sentiment_fl_strtified'+str(f) +'_test.csv',index=False)

fold number :  0
fold number :  1
fold number :  2
fold number :  3
fold number :  4
